## Carga de datos

Antes de poder manipular los diferentes datos, primero es necesário
cargarlos en el sistema

In [31]:
import polars as pl
import numpy as np 
from pathlib import Path
from sklearn.metrics.pairwise import haversine_distances

In [32]:
# Directório de archivos
data_folder = Path("../data")

# Diccionario de archivos
files = {
    "train"              : "train.csv",
    "gas_prices"         : "gas_prices.csv",
    "client"             : "client.csv",
    "electricity_prices" : "electricity_prices.csv",
    "forecast_weather"   : "forecast_weather.csv",
    "historical_weather" : "historical_weather.csv",
    "weather_station"    : "weather_station_to_county_mapping.csv"
}

In [33]:
# Se crea un diccionario de DataFrames para acceder facilmente a todos
# y poder iterar sobre ellos si es necesário
dfs: dict[str, pl.DataFrame] = {}

for key, filename in files.items():
    filepath = data_folder / filename
    try:
        df = pl.read_csv(filepath)
        dfs[key] = df
        print(f"Archivo {key} cargado con exito.")
    except:
        print(f"No pudo cargarse el archivo {key}.")
    

Archivo train cargado con exito.
Archivo gas_prices cargado con exito.
Archivo client cargado con exito.
Archivo electricity_prices cargado con exito.
Archivo forecast_weather cargado con exito.
Archivo historical_weather cargado con exito.
Archivo weather_station cargado con exito.


## Asignación de longitud/latitud a un condado especifico

Con el fin de poder unir los csv de "train.csv" y "forecast_weather.csv"
es necesario poder asignar a cada longitud y latitud, su condado
correspondiente.

Para ello se utilizará el archivo "weather_station_to_county_mapping" con datos que asignan latitudes y longitudes a su correspondiente condado.

### Visualización de los datos

In [212]:
class DataProcessor:
    def __init__(self):
        self.gas_join = ["date"]
        self.ep_join = ["datetime"]
        self.hw_join = [ "county", "datetime"]
        self.client_join = ["product_type", "county", "is_business", "date"]
        self.lat_lon = ["latitude", "longitude"]

    def add_counties(self, df: pl.DataFrame, stations: pl.DataFrame):
        # Se seleccionan los datos requeridos

        coords = df.select(
            pl.col("latitude", "longitude")
        ).unique()

        coords = coords.with_columns(
            pl.col("latitude").radians().alias("lat_rad"),
            pl.col("longitude").radians().alias("lon_rad")
        )

        wthr_stations = stations.drop_nulls()
        wthr_stations = wthr_stations[
            "latitude", "longitude", "county"
        ].with_columns(
                pl.col("latitude", "longitude").radians()
        ).to_numpy()

        coords_arr = coords.to_numpy()

        counties = np.array([], np.int32)
        # Se itera para obtener la menor distancia entre puntos 
        # y asignar el condado
        for coord in coords_arr[:, 2:4]:
            # Cada coordenada tiene una dist minima 
            # y un condado asignado
            min_dist = float("inf")
            cnty = -1

            station_zip = zip(wthr_stations[:, :2], wthr_stations[:, 2])

            for lat_lon, county in station_zip:
                dist = haversine_distances([coord, lat_lon])[0, 1]

                if dist < min_dist:
                    cnty = county
                    min_dist = dist

            # Se añade el condado a la lista
            counties = np.append(
                counties,
                int(cnty)
            )
        
        # Se asigna los condados a cada coordenada y se devuelve el resultado
        result = coords.with_columns(
            pl.lit(counties).alias("county")
        )

        result = df.join(
            result.select(["latitude", "longitude", "county"]),
            on=["latitude", "longitude"],
            how="left"
        )

        return result
    
    def remove_counties(self, df: pl.DataFrame, counties: set[int]):
        return df.filter(~pl.col("county").is_in(counties))

    def change_names(self, df: pl.DataFrame, suffix, no_change):
        renamed_df = df.rename(
            lambda col: col + suffix if col not in no_change else col
        )
        return renamed_df
    
    def to_datetime(self, df: pl.DataFrame, col: str):
        # Cast str to Datetime
        result = df.with_columns(
            pl.col(col).str.strptime(pl.Datetime)
        )
        return result

    def get_gas_features(self, gas_df: pl.DataFrame):
        df = gas_df.with_columns(
            ((pl.col("lowest_price_per_mwh") + 
              pl.col("highest_price_per_mwh")) / 2).alias("mean_price_per_mwh_24h")
        )

        df = df.with_columns(pl.col("origin_date").str.strptime(pl.Date))
        df = df.rename({"origin_date":"date"})

        return self.change_names(df, "_gas", self.gas_join)
    
    def get_client_features(self, client_df: pl.DataFrame):
        df = client_df.with_columns(
            pl.col("date").str.strptime(pl.Date)
        )

        df = self.remove_counties(df, [12])

        return self.change_names(df, "_client", self.client_join)
    
    def get_hw_means(self, data: pl.DataFrame) -> pl.DataFrame:
        data = data.group_by(
        ["datetime", "county"], maintain_order=True
        ).agg(pl.all().mean())
        return data

    def get_hist_weather_features(self, hw_df: pl.DataFrame, ws_df: pl.DataFrame):
        df = self.to_datetime(hw_df, "datetime")

        df = self.add_counties(df, ws_df)
        df = self.get_hw_means(df)
        df = self.change_names(df, "_hw", self.hw_join)

        return df
    
    def predict_epmwh(self, df: pl.DataFrame, date: pl.Datetime):
        from sklearn.linear_model import LinearRegression

        train_df = df.filter(
            pl.col("origin_date").dt.hour() == 2
        )

        X = train_df["data_block_id"].to_numpy().reshape(-1, 1)
        y = train_df["euros_per_mwh"].to_numpy()

        X_pred = train_df.filter(
            pl.col("origin_date") == date - pl.duration(days=1)
        )["data_block_id"].item() + 1
        X_pred = np.array(X_pred).reshape(-1, 1)

        lr = LinearRegression().fit(X, y)
        y_pred = lr.predict(X_pred)

        new_df = pl.DataFrame({
            "forecast_date" : [date + pl.duration(days=1)],
            "euros_per_mwh" : y_pred,
            "origin_date" : [date],
            "data_block_id" : [X_pred.item()]
        })

        result = pl.concat([df, new_df])

        return result

    def get_electricity_features(self, ep_df: pl.DataFrame):
        # str to datetime
        df = self.to_datetime(ep_df, "origin_date")
        df = self.to_datetime(df, "forecast_date")

        df = df.rename({"origin_date"   : "datetime"})
        df = df.rename({"euros_per_mwh" : "euros_per_mwh_24h"})

        # Change names
        df = self.change_names(df, "_ep", self.ep_join)

        return df
    
    def get_data_features(self, data: pl.DataFrame):
        
        date = "datetime"
        # Cast to Datetime
        df = self.to_datetime(data, "datetime")

        df = df.with_columns([
            # Date
            pl.col(date).dt.truncate("1d").cast(pl.Date).alias("date"),

            # Year
            pl.col(date).dt.year().alias("year"),

            # Quarter
            pl.col(date).dt.quarter().alias("quarter"),

            # Month
            pl.col(date).dt.month().alias("month"),

            # Week
            pl.col(date).dt.week().alias("week"),

            # Hour
            pl.col(date).dt.hour().alias("hour"),

            # Day of year
            pl.col(date).dt.ordinal_day().alias("day_of_year"),

            # Day of month
            pl.col(date).dt.day().alias("day_of_month"),

            # Day of week
            pl.col(date).dt.weekday().alias("day_of_week")
        ])

        df = self.remove_counties(df, [12])

        return df
    
    def __call__(self, df_dict: dict):

        # Features
        data = self.get_data_features(df_dict["train"])
        gas_p = self.get_gas_features(df_dict["gas_prices"])
        el_p = self.get_electricity_features(df_dict["electricity_prices"])
        client = self.get_client_features(df_dict["client"])
        hist_w = self.get_hist_weather_features(
            df_dict["historical_weather"],
            df_dict["weather_station"]
        )

        # Joins
        final_data = data.join(client, on=self.client_join, how="left")
        final_data = final_data.join(gas_p, on=self.gas_join, how="left")
        final_data = final_data.join(el_p, on=self.ep_join, how="left")
        final_data = final_data.join(hist_w, on=self.hw_join, how="left")

        limit_date = pl.lit("2023-05-29 23:00:00").str.strptime(pl.Datetime)

        final_data = final_data.filter(pl.col("datetime") <= limit_date)

        return final_data

In [ ]:
dp = DataProcessor()
res = dp(dfs)

In [36]:
res

county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,year,quarter,month,week,hour,day_of_year,day_of_month,day_of_week,eic_count_client,installed_capacity_client,data_block_id_client,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,data_block_id_gas,mean_price_per_mwh_24h_gas,forecast_date_ep,euros_per_mwh_24h_ep,data_block_id_ep,temperature_hw,dewpoint_hw,rain_hw,snowfall_hw,surface_pressure_hw,cloudcover_total_hw,cloudcover_low_hw,cloudcover_mid_hw,cloudcover_high_hw,windspeed_10m_hw,winddirection_10m_hw,shortwave_radiation_hw,direct_solar_radiation_hw,diffuse_radiation_hw,latitude_hw,longitude_hw,data_block_id_hw
i64,i64,i64,f64,i64,datetime[μs],i64,i64,i64,date,i32,i8,i8,i8,i8,i16,i8,i8,i64,f64,i64,str,f64,f64,i64,f64,str,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0,1,0.713,0,2021-09-01 00:00:00,0,0,0,2021-09-01,2021,3,9,35,0,244,1,3,108,952.89,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,1,96.59,1,2021-09-01 00:00:00,0,1,0,2021-09-01,2021,3,9,35,0,244,1,3,108,952.89,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,2,0.0,0,2021-09-01 00:00:00,0,2,1,2021-09-01,2021,3,9,35,0,244,1,3,17,166.4,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,2,17.314,1,2021-09-01 00:00:00,0,3,1,2021-09-01,2021,3,9,35,0,244,1,3,17,166.4,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
0,0,3,2.904,0,2021-09-01 00:00:00,0,4,2,2021-09-01,2021,3,9,35,0,244,1,3,688,7207.88,2,"""2021-09-02""",45.62,46.29,2,45.955,"""2021-09-02 00:00:00""",96.99,2,13.815385,10.6,0.0,0.0,1011.569231,42.461538,44.153846,2.923077,3.384615,5.619658,341.384615,0.0,0.0,0.0,59.492308,24.584615,1.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
15,1,0,188.167,1,2023-05-29 23:00:00,635,2012107,64,2023-05-29,2023,2,5,22,23,149,29,1,15,620.0,637,"""2023-05-30""",29.0,34.0,637,31.5,"""2023-05-30 23:00:00""",-1.29,637,11.72,7.02,0.0,0.0,1006.16,19.8,2.1,15.5,28.8,1.802778,173.1,0.0,0.0,0.0,57.78,27.4,637.0
15,1,1,0.0,0,2023-05-29 23:00:00,635,2012108,59,2023-05-29,2023,2,5,22,23,149,29,1,20,624.5,637,"""2023-05-30""",29.0,34.0,637,31.5,"""2023-05-30 23:00:00""",-1.29,637,11.72,7.02,0.0,0.0,1006.16,19.8,2.1,15.5,28.8,1.802778,173.1,0.0,0.0,0.0,57.78,27.4,637.0
15,1,1,31.484,1,2023-05-29 23:00:00,635,2012109,59,2023-05-29,2023,2,5,22,23,149,29,1,20,624.5,637,"""2023-05-30""",29.0,34.0,637,31.5,"""2023-05-30 23:00:00""",-1.29,637,11.72,7.02,0.0,0.0,1006.16,19.8,2.1,15.5,28.8,1.802778,173.1,0.0,0.0,0.0,57.78,27.4,637.0


In [47]:
res.describe()

statistic,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,date,year,quarter,month,week,hour,day_of_year,day_of_month,day_of_week,eic_count_client,installed_capacity_client,data_block_id_client,forecast_date_gas,lowest_price_per_mwh_gas,highest_price_per_mwh_gas,data_block_id_gas,mean_price_per_mwh_24h_gas,forecast_date_ep,euros_per_mwh_24h_ep,data_block_id_ep,temperature_hw,dewpoint_hw,rain_hw,snowfall_hw,surface_pressure_hw,cloudcover_total_hw,cloudcover_low_hw,cloudcover_mid_hw,cloudcover_high_hw,windspeed_10m_hw,winddirection_10m_hw,shortwave_radiation_hw,direct_solar_radiation_hw,diffuse_radiation_hw,latitude_hw,longitude_hw,data_block_id_hw
str,f64,f64,f64,f64,f64,str,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",1.981584e6,1.981584e6,1.981584e6,1.981064e6,1.981584e6,"""1981584""",1.981584e6,1.981584e6,1.981584e6,"""1981584""",1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,"""1981584""",1.981584e6,1.981584e6,1.981584e6,1.981584e6,"""1981322""",1.981322e6,1.981322e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6,1.981584e6
"""null_count""",0.0,0.0,0.0,520.0,0.0,"""0""",0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""0""",0.0,0.0,0.0,0.0,"""262""",262.0,262.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",7.224645,0.529637,1.882034,274.42834,0.5,"""2022-07-19 10:19:47.050359""",320.951239,1.0062e6,32.797641,"""2022-07-18 22:49:47.050000""",2022.05409,2.479786,6.434658,26.138604,11.5,180.208367,15.655645,4.002761,74.317177,1467.247208,322.951239,null,95.53234,108.393963,322.951239,101.963152,null,157.514057,322.942557,5.443399,1.917484,0.048351,0.016764,1007.948216,62.03052,47.952422,35.104989,36.121579,4.468892,198.376072,105.425972,62.9162,42.509772,58.63522,25.54854,322.492906
"""std""",4.781334,0.499121,1.081267,915.138469,0.5,null,182.049529,580772.194083,19.637244,null,0.643975,1.180241,3.669624,15.95755,6.922188,112.110155,8.760926,2.000108,144.953325,2436.993209,182.049529,null,47.570707,54.742788,182.049529,50.915722,null,121.419656,182.047971,8.170889,7.240976,0.175282,0.069765,12.505641,35.944698,38.982885,36.439033,39.267133,2.045021,82.535344,175.151736,127.935893,61.510183,0.588587,1.631624,182.050211
"""min""",0.0,0.0,0.0,0.0,0.0,"""2021-09-01 00:00:00""",0.0,0.0,0.0,"""2021-09-01""",2021.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,5.0,5.5,2.0,"""2021-09-02""",28.1,34.0,2.0,31.1,"""2021-09-02 00:00:00""",-10.06,2.0,-20.533333,-23.366667,0.0,0.0,952.9,0.0,0.0,0.0,0.0,0.166667,1.0,0.0,0.0,0.0,57.78,22.144444,1.0
"""25%""",3.0,0.0,1.0,0.356,0.0,"""2022-02-13 19:00:00""",165.0,503354.0,16.0,"""2022-02-13""",2022.0,1.0,3.0,12.0,6.0,80.0,8.0,2.0,14.0,324.0,167.0,null,60.0,67.67,167.0,65.0,null,85.29,167.0,-0.475,-3.02,0.0,0.0,1000.738462,28.153846,6.333333,0.75,0.0,2.909722,144.0,0.0,0.0,0.0,58.094118,24.533333,167.0
"""50%""",7.0,1.0,2.0,30.963,1.0,"""2022-07-20 12:00:00""",322.0,1.006272e6,32.0,"""2022-07-20""",2022.0,2.0,6.0,24.0,12.0,167.0,16.0,4.0,32.0,657.0,324.0,null,86.0,94.0,324.0,88.895,null,128.79,324.0,4.966667,1.433333,0.0,0.0,1009.015385,73.2,44.8,21.0,16.0,4.242424,206.777778,6.133333,0.0,4.4,58.725,25.533333,324.0
"""75%""",11.0,1.0,3.0,174.627,1.0,"""2022-12-23 03:00:00""",478.0,1.509137e6,50.0,"""2022-12-23""",2022.0,4.0,10.0,41.0,17.0,285.0,23.0,6.0,71.0,1574.9,480.0,null,109.74,133.0,480.0,120.91,null,199.99,480.0,10.966667,6.8,0.007692,0.0,1016.222222,97.666667,90.333333,68.454545,78.9,5.735043,257.333333,138.75,47.733333,74.666667,59.233333,27.116667,479.0
"""max""",15.0,1.0,3.0,15480.274,1.0,"""2023-05-29 23:00:00""",635.0,2.012111e6,68.0,"""2023-05-29""",2023.0,4.0,12.0,52.0,23.0

In [213]:
dp2 = DataProcessor()

ep = dfs["electricity_prices"].with_columns(
    pl.col("origin_date").str.strptime(pl.Datetime),
    pl.col("forecast_date").str.strptime(pl.Datetime)
)

day = pl.lit("2022-03-26 02:00:00").str.strptime(pl.Datetime)

result = dp2.predict_epmwh(ep, day)
result

SchemaError: type Object("object", Some(object-registry)) is incompatible with expected type Datetime(Microseconds, None)

In [179]:
res.filter(pl.col("euros_per_mwh_24h_ep").is_null())["datetime"].value_counts()

datetime,count
datetime[μs],u32
2022-03-26 02:00:00,132
2023-03-25 02:00:00,130


In [201]:
guess_2am = dfs["electricity_prices"].with_columns(
    pl.col("origin_date").str.strptime(pl.Datetime),
    pl.col("origin_date").str.strptime(pl.Datetime).dt.date().alias("date"),
    pl.col("origin_date").str.strptime(pl.Datetime).dt.hour().alias("hour")
)

In [202]:
guess_2am.head()

forecast_date,euros_per_mwh,origin_date,data_block_id,date,hour
str,f64,datetime[μs],i64,date,i8
"""2021-09-01 00:00:00""",92.51,2021-08-31 00:00:00,1,2021-08-31,0
"""2021-09-01 01:00:00""",88.9,2021-08-31 01:00:00,1,2021-08-31,1
"""2021-09-01 02:00:00""",87.35,2021-08-31 02:00:00,1,2021-08-31,2
"""2021-09-01 03:00:00""",86.88,2021-08-31 03:00:00,1,2021-08-31,3
"""2021-09-01 04:00:00""",88.43,2021-08-31 04:00:00,1,2021-08-31,4


In [208]:
dt_gs = pl.lit("2021-08-31 02:00:00").str.strptime(pl.Datetime)
guess_2am.filter(
    pl.col("origin_date") == dt_gs + pl.duration(days=1)
)["data_block_id"].item()

2

In [196]:
a = np.array([1, 2, 3, 4]).reshape(-1, 1)
a[-1] + 1, a[-1]

(array([5]), array([4]))

In [174]:
guess_2am.filter(
    pl.col("hour") == 2,
    pl.col("date") == pl.lit("2023-03-25").str.strptime(pl.Date)
)

ColumnNotFoundError: unable to find column "hour"; valid columns: ["forecast_date", "euros_per_mwh", "origin_date", "data_block_id"]

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'filter' <---
DF ["forecast_date", "euros_per_mwh", "origin_date", "data_block_id"]; PROJECT */4 COLUMNS

In [48]:
'''for key in dfs.keys():
    print(key)
    display(dfs[key].describe())
'''

'for key in dfs.keys():\n    print(key)\n    display(dfs[key].describe())\n'

In [49]:

'''for key in dfs.keys():
    print(f"{key}:")
    display(dfs[key].head(1))
'''

'for key in dfs.keys():\n    print(f"{key}:")\n    display(dfs[key].head(1))\n'